In [1]:
# !wget https://huggingface.co/datasets/syafie-nzm/crawl-jurnaldbp/resolve/main/jurnaldbp.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/melayu-pdf.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/crossref-pdf.jsonl

In [2]:
from glob import glob
from tqdm import tqdm
import json
import os
import numpy as np
from transformers import AutoTokenizer
from streaming import MDSWriter
from tqdm import tqdm
import msgspec
import json
import re

http_errors = [
        "400 Bad Request", "401 Unauthorized", "402 Payment Required", "403 Forbidden", "404 Not Found",
        "405 Method Not Allowed", "406 Not Acceptable", "407 Proxy Authentication Required", "408 Request Timeout",
        "409 Conflict", "410 Gone", "411 Length Required", "412 Precondition Failed", "413 Payload Too Large",
        "414 URI Too Long", "415 Unsupported Media Type", "416 Range Not Satisfiable", "417 Expectation Failed",
        "418 I'm a teapot", "421 Misdirected Request", "422 Unprocessable Entity", "423 Locked", "424 Failed Dependency",
        "425 Too Early", "426 Upgrade Required", "428 Precondition Required", "429 Too Many Requests",
        "431 Request Header Fields Too Large", "451 Unavailable For Legal Reasons", "500 Internal Server Error",
        "501 Not Implemented", "502 Bad Gateway", "503 Service Unavailable", "504 Gateway Timeout",
        "505 HTTP Version Not Supported", "506 Variant Also Negotiates", "507 Insufficient Storage",
        "508 Loop Detected", "510 Not Extended", "511 Network Authentication Required"
    ]

rejected = [
    'Internal Server Error',
    '__NOEDITSECTION__',
    'enter your username and password',
    'forgotten your password',
    'cookies enabled',
    'enable JavaScript in your browser.',
    'The page cannot be displayed',
    'site or edit the error_page',
    'Request unsuccessful',
]

rejected.extend(http_errors)

def replace_multiple(input_string, pattern =r"\s{6,}", replace = '   '):
    return re.sub(pattern, replace, input_string)

def replace(string):
    string = replace_multiple(string.replace('…', '.'))
    string = replace_multiple(string, pattern = r"\.{6,}", replace = '...')
    return string

def reject(string):
    if any([r in string for r in rejected]):
        return True
    return False

In [3]:
files = [
    'jurnaldbp.jsonl',
    'melayu-pdf.jsonl',
]

In [4]:
def partition(text, size = 500):
    splitted = text.split()
    return [' '.join(splitted[i: i + size]) for i in range(0, len(splitted), size)]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('upstage/SOLAR-10.7B-v1.0')

In [7]:
a = open('combine-lm.jsonl', 'w')

In [8]:
for f in files:
    with open(f, encoding = "ISO-8859-1") as fopen:
        for l in tqdm(fopen):
            try:
                l = msgspec.json.decode(l)
                if reject(l):
                    continue

                l = replace(l.strip())
                if len(l) < 3:
                    continue

                data = '<s>' + l + '</s>'
                partitioned = partition(data)
                for p in partitioned:
                    data = {
                        'text': p,
                    }
                    a.write(f'{json.dumps(data)}\n')
                    a.flush()
            except Exception as e:
                print(e)
                pass

930it [00:02, 317.00it/s]
1506it [00:10, 140.25it/s]


In [9]:
files = glob('/home/ubuntu/dedup-text-dataset/*.jsonl')
len(files)

274

In [11]:
wiki = [
    'wikipedia-2023-10-01.jsonl',
    'wikipedia-jawi.jsonl',
]
language_related = [
    'dictionary.jsonl',
    'dewanbahasa-jdbp.jsonl',
    'dialect.jsonl',
    'kamusbm.jsonl',
]
gov_related = [
    'hansard.jsonl',
    'lom.agc.gov.my.jsonl',
    'parlimen-gov.jsonl',
    'data.gov.my.jsonl',
    'mufti_wilayah_articles.jsonl',
    'e-khutbah.jsonl',
    'mufti_negeri_sem_artikel.jsonl',
    'mufti_perlis_artikel.jsonl',
    'mufti_negeri_sem_artikel.jsonl',
    'gov.my.jsonl',
    'edu.my.jsonl',
]
research_papers = [
    'academia-edu.jsonl',
    'eprints.jsonl',
]
social_media = [
    'iium-confession.jsonl',
    'b.cari.com.my.jsonl',
    'semisupervised-whisper-large-v2.jsonl',
    'lowyat.jsonl',
    'malay-tweets.jsonl',
    'c.cari.com.my.jsonl',
    'cn.cari.com.my.jsonl',
    'carigold.jsonl'
]
common_crawl = [
    'common-crawl.jsonl',
    'NLLB.jsonl',
]
buku_teks = [
    'buku-teks.jsonl',
    'bumigemilang.com.jsonl',
    'tcer.my.jsonl',
    'mysoalan.com.jsonl'
]

In [12]:
combine = set(wiki) | set(language_related) | set(gov_related) | set(research_papers) | set(common_crawl) | set(buku_teks)

In [13]:
combine = {os.path.join('/home/ubuntu/dedup-text-dataset', f) for f in combine}

In [14]:
online_articles = sorted(list(set(files) - combine))
len(online_articles)

250

In [15]:
rejected = social_media + common_crawl + [
    'pdfdrive.jsonl',
    'seehua.jsonl',
    'hardwarezone-sg.jsonl',
    'sinchew.com.my.jsonl',
    'orientaldaily.com.my.jsonl'
]

In [16]:
online_articles = [x for x in online_articles if 'c4-filtered' not in x and 'the-pile' not in x and 'c.cari.com.my' not in x]

In [17]:
online_articles = [x for x in online_articles if all([r not in x for r in rejected])]
len(online_articles)

235

In [18]:
for f in wiki:
    f = os.path.join('/home/ubuntu/dedup-text-dataset', f)
    with open(f) as fopen:
        for l in tqdm(fopen):
            try:
                
                data = '<s>' + json.loads(l) + '</s>'
                partitioned = partition(data)
                for p in partitioned:
                    data = {
                        'text': p,
                    }
                    a.write(f'{json.dumps(data)}\n')
                    a.flush()
            except:
                pass

438316it [00:12, 35416.45it/s]
722837it [00:18, 39232.31it/s]


In [19]:
for f in language_related:
    f = os.path.join('/home/ubuntu/dedup-text-dataset', f)
    with open(f) as fopen:
        for l in tqdm(fopen):
            try:
                data = '<s>' + json.loads(l) + '</s>'
                partitioned = partition(data)
                for p in partitioned:
                    data = {
                        'text': p,
                    }
                    a.write(f'{json.dumps(data)}\n')
                    a.flush()
            except:
                pass

54712it [00:05, 10045.04it/s]
4577it [00:00, 12655.12it/s]
66it [00:00, 52112.96it/s]
34192it [00:01, 29373.62it/s]


In [20]:
for f in buku_teks:
    f = os.path.join('/home/ubuntu/dedup-text-dataset', f)
    with open(f) as fopen:
        for l in tqdm(fopen):
            try:
                data = '<s>' + json.loads(l) + '</s>'
                partitioned = partition(data)
                for p in partitioned:
                    data = {
                        'text': p,
                    }
                    a.write(f'{json.dumps(data)}\n')
                    a.flush()
            except:
                pass


183it [00:00, 277.55it/s]
16120it [00:09, 1652.72it/s]
3625it [00:00, 5755.95it/s]
900it [00:00, 4812.70it/s]


In [21]:
for f in gov_related:
    f = os.path.join('/home/ubuntu/dedup-text-dataset', f)
    with open(f) as fopen:
        for l in tqdm(fopen):
            try:
                data = '<s>' + json.loads(l) + '</s>'
                partitioned = partition(data)
                for p in partitioned:
                    data = {
                        'text': p,
                    }
                    a.write(f'{json.dumps(data)}\n')
                    a.flush()
            except:
                pass

140932it [00:15, 9290.93it/s] 
1359it [00:02, 538.86it/s]
1687it [00:07, 238.33it/s]
10889it [00:18, 586.85it/s]
1712it [00:00, 5882.96it/s]
809it [00:00, 3069.10it/s]
112it [00:00, 22988.11it/s]
144it [00:00, 19651.84it/s]
112it [00:00, 22658.79it/s]
30055it [00:33, 891.29it/s] 
21590it [00:21, 1000.28it/s]


In [22]:
rejected = [
    'markah untuk setiap satu',
    'soalan mesti dijawab dalam',
    '25 markah',
    '50 markah'
]

In [23]:
for f in research_papers:
    f = os.path.join('/home/ubuntu/dedup-text-dataset', f)
    with open(f) as fopen:
        for l in tqdm(fopen):
            try:
                data = '<s>' + json.loads(l) + '</s>'
                    
                data_lower = data.lower()
                if any([r in data_lower for r in rejected]):
                    continue
                    
                partitioned = partition(data)
                for p in partitioned:
                    data = {
                        'text': p,
                    }
                    a.write(f'{json.dumps(data)}\n')
                    a.flush()
            except:
                pass

787it [00:06, 115.49it/s]
189419it [03:07, 1008.36it/s]


In [24]:
for f in online_articles:
    with open(f) as fopen:
        for l in tqdm(fopen):
            try:
                data = '<s>' + json.loads(l) + '</s>'
                partitioned = partition(data)
                for p in partitioned:
                    data = {
                        'text': p,
                    }
                    a.write(f'{json.dumps(data)}\n')
                    a.flush()
            except:
                pass

14954it [00:00, 29172.37it/s]
752it [00:00, 5758.37it/s]
172it [00:00, 24873.13it/s]
20763it [00:01, 17446.08it/s]
640it [00:00, 10970.79it/s]
74it [00:00, 3850.75it/s]
10604it [00:00, 21896.72it/s]
1733it [00:00, 14037.69it/s]
33730it [00:01, 31879.12it/s]
144it [00:00, 9951.06it/s]
359it [00:00, 9476.42it/s]
87it [00:00, 13378.71it/s]
47it [00:00, 9812.95it/s]
1263it [00:00, 6970.54it/s]
9672it [00:00, 13420.11it/s]
5400it [00:00, 17883.55it/s]
4425it [00:00, 23536.55it/s]
269it [00:00, 424.90it/s]
3645it [00:01, 3426.56it/s]
1589it [00:00, 9482.07it/s]
385it [00:00, 10950.29it/s]
17004it [00:08, 2123.16it/s]
176it [00:00, 6920.26it/s]
233it [00:00, 10569.23it/s]
48it [00:00, 9486.25it/s]
1427it [00:00, 8055.66it/s]
341it [00:00, 4279.33it/s]
1546it [00:01, 1528.89it/s]
46it [00:00, 13109.87it/s]
140it [00:00, 15868.20it/s]
11678it [00:01, 10213.22it/s]
12326it [00:01, 6540.96it/s] 
24482it [00:02, 8486.05it/s] 
947it [00:00, 6098.52it/s]
39678991it [07:37, 86708.19it/s] 
512it [00:0

In [26]:
!ls -lh combine-lm.jsonl

-rw-r--r-- 1 ubuntu ubuntu 38G Dec 28 08:43 combine-lm.jsonl
